In [ ]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
#train_data = pd.read_csv('/content/train.txt', sep = ';')
#train_data.head()

In [ ]:
def preprocess(file):
    path = '/content/'+file+'.txt'
    data = pd.read_csv(path, sep = ';')
    hos = []
    for i in range(len(data.emotion)):
        if data['emotion'][i] in ['joy', 'love', 'surprise']:
            hos.append(1) # happy is 1
        else:
            hos.append(0) # sad is 0
    data['hos'] = hos
    return data

In [ ]:
train_data = preprocess('train')
train = train_data.copy()

In [ ]:
model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(model, output_shape=[20], input_shape=[], dtype=tf.string, trainable=True)

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [ ]:
model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.0, name='accuracy')])

In [ ]:
val = preprocess('val')

In [ ]:
history = model.fit(train.text,
                    train.hos,
                    epochs=40,
                    batch_size=512,
                    validation_data=(val.text, val.hos),
                    verbose = 0)

In [ ]:
test = preprocess('test')
results = model.evaluate(test['text'], test['hos'])

print(results)

1/1 [==============================] - 0s 29ms/step - loss: 0.0542 - accuracy: 1.0000
[0.05420124903321266, 1.0]


In [ ]:
predstr = model.predict(train.text)

500/500 [==============================] - 1s 2ms/step


In [ ]:
def postprocessor(preds):
  range = predstr.max()-predstr.min()
  norm_preds = []
  probab = []
  for i in preds:
    norm_preds.append((i - predstr.min()) / range)
    probab.append((i - predstr.min()) * 100 / range)
  return np.mean(probab)#, probab

In [ ]:
answers = []
answers.append(input('How would you describe your experience at your workplace/college/school in the past few days?'))
answers.append(input('How do you like to spend your leisure time? How do you feel after it?'))
answers.append(input('Life has its ups and downs. Although handling successes can be difficult, setbacks can affect mental health strongly. How do you manage your emotions after failures?'))
#answers.append(input('Any recent pleasant experience that you would like to share?'))
#answers.append(input('Any recent unpleasant experience that you would like to share?'))
answers.append(input('In a broad sense, how would you describe the way your life is going on?'))
#answers.append(input('How would you describe your experience at your workplace/college/school in the past few days?'))
results = model.predict(answers)
print('Your mental health score is:', postprocessor(results))